In [2]:
import glob
import multiprocessing
import os
import re
import sys
import tempfile
from functools import lru_cache
from pathlib import Path
from zipfile import ZipFile

from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from rapidfuzz import fuzz, process
from tqdm import tqdm

import polars as pl
pl.Config.set_fmt_str_lengths(100)

sys.path.append('..')
from src.read_iris import read_iris


/home/leo/.local/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


### IRIS DFs

In [3]:
df_iris_master = pl.read_csv('data/iris-data-2024-03-14/ODS_L1_IR_ITEM_MASTER_ALL.csv', columns=['ITEM_ID', 'YEAR_PUBLISHED', 'TITLE'] ,dtypes={'ITEM_ID': pl.Int32, 'YEAR_PUBLISHED': pl.Utf8, 'TITLE': pl.Utf8})
df_iris_identifier = pl.read_csv('data/iris-data-2024-03-14/ODS_L1_IR_ITEM_IDENTIFIER.csv', columns=['ITEM_ID', 'IDE_DOI', 'IDE_ISBN', 'IDE_PMID'] ,dtypes={'ITEM_ID': pl.Int32, 'IDE_DOI': pl.Utf8, 'IDE_ISBN': pl.Utf8, 'IDE_PMID': pl.Utf8})

df = df_iris_identifier.join(df_iris_master, on='ITEM_ID', how='inner')


df_filtered = df.filter(pl.col('IDE_DOI').is_not_null() | pl.col('IDE_ISBN').is_not_null() | pl.col('IDE_PMID').is_not_null())[
    ['ITEM_ID', 'IDE_DOI', 'IDE_ISBN', 'IDE_PMID']
]

df_filtered.head()

ITEM_ID,IDE_DOI,IDE_ISBN,IDE_PMID
i32,str,str,str
2766,"""10.14411/fp.2008.015""",null,"""18666414"""
3349,"""10.1016/j.econlet.2011.12.096""",null,null
3867,null,"""9788876655746""",null
4342,null,"""8821724038""",null
4585,null,"""9788861328112""",null


In [11]:
len(df_filtered)

201471

In [12]:
df.nuni

statistic,ITEM_ID,IDE_DOI,IDE_ISBN,IDE_PMID,YEAR_PUBLISHED,TITLE
str,f64,str,str,str,str,str
"""count""",304952.0,"""131048""","""81526""","""45855""","""304950""","""304948"""
"""null_count""",0.0,"""173904""","""223426""","""259097""","""2""","""4"""
"""mean""",325525.479262,null,null,null,null,null
"""std""",236254.328994,null,null,null,null,null
"""min""",13.0,"""(doi: 10.1412/36459)""","""""8883424352; 9788883424359""","""0002541535""","""1967""",""" Canine Diabetes Mellitus"""
"""25%""",126416.0,null,null,null,null,null
"""50%""",236823.0,null,null,null,null,null
"""75%""",535059.0,null,null,null,null,null
"""max""",816910.0,"""https://zenodo.org/communities/enti/""","""­""","""Raimondo D, Maletta M, Borghese G, Mastronardi M, Arena A, Del Forno S, Salucci P, Giaquinto I, Cas…","""9999""","""￼￼New Trends in Databases and Information Systems"""


---
### Sanitize IRIS DOIs and ISBNs

In [12]:
dois = df_filtered.select('ITEM_ID', 'IDE_DOI').drop_nulls('IDE_DOI').unique()

filtered_dois = (
    dois
    .with_columns(('doi:'+pl.col('IDE_DOI').str.extract(r'(10\.\d{4,}\/[^,\s;]*)').str.to_lowercase()).alias('id'))
    .drop_nulls('id')
    .drop('IDE_DOI')
    .rename({'ITEM_ID': 'iris_id'})
)

filtered_dois

iris_id,id
i32,str
217903,"""doi:10.1109/eumic.2014.6997826"""
271797,"""doi:10.1016/j.jvolgeores.2016.11.016"""
380360,"""doi:10.1051/0004-6361/201832843"""
698521,"""doi:10.1080/14737159.2020.1699791"""
627693,"""doi:10.1007/s00170-021-08035-w"""
…,…
786885,"""doi:10.1007/jhep05(2023)233"""
627207,"""doi:10.3174/ajnr.a6892"""
252488,"""doi:10.1111/clr.13_12679"""


In [13]:
isbns = df_filtered.select('ITEM_ID', 'IDE_ISBN').drop_nulls('IDE_ISBN').unique()

filtered_isbns = (
    isbns
    .with_columns(
        ('isbn:'+pl.col('IDE_ISBN').str.extract_all(r'(ISBN[-]*(1[03])*[ ]*(: ){0,1})*(([0-9Xx][- ]*){13}|([0-9Xx][- ]*){10})').list.first().str.replace_all(r'[- ]', '').str.to_lowercase()).alias('id')
        #('doi:'+pl.col('IDE_DOI')).alias('id')
    )
    .drop_nulls('id')
    .drop('IDE_ISBN')
    .rename({'ITEM_ID': 'iris_id'})
)

filtered_isbns

iris_id,id
i32,str
128597,"""isbn:9788861295667"""
78543,"""isbn:8815114254"""
181746,"""isbn:9788888793481"""
798185,"""isbn:9788893592956"""
345364,"""isbn:9781407316307"""
…,…
241284,"""isbn:9788867740932"""
117011,"""isbn:9788495886392"""
84622,"""isbn:8884833140"""


In [14]:
pmids = df_filtered.select('ITEM_ID', 'IDE_PMID').drop_nulls('IDE_PMID').unique()

filtered_pmids = (
    pmids
    .filter(
        ~pl.col('IDE_PMID').str.contains('PMC')
        )
    .with_columns(('pmid:'+pl.col('IDE_PMID').str.extract(r'0*([1-9][0-9]{1,8})', 1).str.to_lowercase()).alias('id'))
    .drop('IDE_PMID')
    .rename({'ITEM_ID': 'iris_id'})
)

filtered_pmids

iris_id,id
i32,str
311595,"""pmid:27062486"""
762705,"""pmid:36979644"""
323738,"""pmid:29172868"""
804454,"""pmid:37651845"""
806067,"""pmid:22577222"""
…,…
111642,"""pmid:19194070"""
574944,"""pmid:33924854"""
175443,"""pmid:24769871"""


In [15]:
dois_isbns_pmids_df = pl.concat([filtered_dois, filtered_isbns, filtered_pmids])

dois_isbns_pmids_df

iris_id,id
i32,str
217903,"""doi:10.1109/eumic.2014.6997826"""
271797,"""doi:10.1016/j.jvolgeores.2016.11.016"""
380360,"""doi:10.1051/0004-6361/201832843"""
698521,"""doi:10.1080/14737159.2020.1699791"""
627693,"""doi:10.1007/s00170-021-08035-w"""
…,…
111642,"""pmid:19194070"""
574944,"""pmid:33924854"""
175443,"""pmid:24769871"""


##### old

In [16]:
dois = df_filtered.select('IDE_DOI').drop_nulls().unique()['IDE_DOI'].to_list()
print(len(dois), 'unique dois')
dois_q = df_filtered.select('ITEM_ID', 'IDE_DOI').drop_nulls('IDE_DOI').filter(pl.col('IDE_DOI').str.starts_with('10.1007/jhep02(2013)043'))

#filter and normalize the dois
doi_rule = re.compile(r'10\.\d{4,}\/[^,\s;]*')
not_doi = []
filtered_dois = []

for doi in dois:
    match = doi_rule.search(doi)
    if match:
        filtered_dois.append('doi:' + match.group())
    else:
        not_doi.append(doi)

print(len(filtered_dois), 'unique dois after filtering. ', len(not_doi), 'not dois discarded')

print('---'*10)

isbns = df_filtered.select('IDE_ISBN').drop_nulls().unique()['IDE_ISBN'].to_list()
print(len(isbns), 'unique isbns')

#filter and normalize the isbns
isbn_rule = re.compile(r'(ISBN[-]*(1[03])*[ ]*(: ){0,1})*(([0-9Xx][- ]*){13}|([0-9Xx][- ]*){10})') # ??? results to check
not_isbn = []
filtered_isbns = []

for isbn in isbns:
    if isbn_rule.search(isbn) is not None:
        filtered_isbns.append('isbn:' + isbn.replace('-', '').replace(' ', ''))
    else:
        not_isbn.append(isbn)

print(len(filtered_isbns), 'unique isbns after filtering. ', len(not_isbn), 'not isbns discarded')

print('---'*10)

#filter and normalize the pmids
pmids = df_filtered.select('IDE_PMID').drop_nulls().unique()['IDE_PMID'].to_list()
pmid_rule = re.compile(r'(?!0\d{0,7})\d{1,8}')
not_pmids = []
filtered_pmids = []

for pmid in pmids:
    match = pmid_rule.search(pmid)
    if pmid_rule.search(pmid) is not None and 'PMC' not in pmid: # questo da sottolineare in documentazione
        filtered_pmids.append('pmid:' + match.group())
    else:
        not_pmids.append(pmid)

print(len(pmids), 'unique pmids')
print(len(filtered_pmids), 'unique pmids after filtering. ', len(not_pmids), 'not pmids discarded')

dois_isbns_pmids = filtered_dois + filtered_isbns + filtered_pmids
dois_isbns_pmids = [id.lower() for id in dois_isbns_pmids]

print('==='*10)
print(len(dois_isbns_pmids), 'total identifiers')

130077 unique dois
129954 unique dois after filtering.  123 not dois discarded
------------------------------
49982 unique isbns
49405 unique isbns after filtering.  577 not isbns discarded
------------------------------
45687 unique pmids
45683 unique pmids after filtering.  4 not pmids discarded
225042 total identifiers


---
### Meta to Parquet

In [94]:
meta_path = Path('data/csv_openalex.zip')

In [104]:
def process_meta_zip(zip_path):
    zip_file = ZipFile(zip_path)
    files_list = [zipfile for zipfile in zip_file.namelist() if zipfile.endswith('.csv')]
    output_dir = "data/iris_in_meta_join"

    dois_isbns_pmids_lf = dois_isbns_pmids_df.lazy()

    for csv_file in tqdm(files_list):
        with zip_file.open(csv_file, 'r') as file:
            with tempfile.NamedTemporaryFile() as tf:
                tf.write(file.read()) # Write the csv file to the temporary file
                tf.seek(0)          # Start at the beginning of the temporary file
                os.makedirs(output_dir, exist_ok=True)
                df = (
                    pl.scan_csv(tf.name)
                    .select(['id', 'title', 'type'])
                    .with_columns(
                        #(pl.col('id').str.extract(r"((?:isbn):[^\s]+)")).alias('isbn'),
                        (pl.col('id').str.extract(r"(omid:[^\s]+)")).alias('omid'),
                        # if it has a doi we get the doi, otherwise we get the isbn - the right approach?
                        (pl.col('id').str.extract(r"((?:doi):[^\s\"]+)")).alias('doi'),
                        (pl.col('id').str.extract(r"((?:pmid):[^\s\"]+)")).alias('pmid'),
                        (pl.col('id').str.extract(r"((?:isbn):[^\s\"]+)")).alias('isbn'),
                    )
                    .with_columns(
                        pl.coalesce([pl.col('doi'), pl.col('isbn'), pl.col('pmid')]).alias('id')
                    )
                    .drop(['doi', 'pmid', 'isbn'])
                    .drop_nulls('id')
                    .join(dois_isbns_pmids_lf, on='id', how='inner')
                    .collect(streaming=True)
                )

                if not df.is_empty():
                    df.write_parquet(os.path.join(output_dir, os.path.basename(csv_file).replace('.csv', '.parquet')))


process_meta_zip(meta_path) # 35mins

100%|██████████| 28248/28248 [13:14<00:00, 35.55it/s]


### Title similarity

#### sparql

In [5]:
iris_noid_titles = (
    df
    .select('ITEM_ID', 'IDE_DOI', 'IDE_ISBN', 'IDE_PMID', 'TITLE')
    .filter(
        (pl.col('IDE_DOI').is_null() & pl.col('IDE_ISBN').is_null() & pl.col('IDE_PMID').is_null()),
    )
    .drop('IDE_DOI', 'IDE_ISBN', 'IDE_PMID')
)#['TITLE'].to_list()

len(iris_noid_titles)
#iris_noid_titles_clean = [tile.replace('"', "'") for title in iris_noid_titles if title is not None and len(title.split()) > 3]
#len(iris_noid_titles_clean)
iris_noid_titles

ITEM_ID,TITLE
i32,str
4837,"""Mutational spectrum of SHOX gene in 25 Italian pediatric patients with Lèri-Weill dyschondrosteosis…"
5279,"""Operational Fiscal and Monetary Policy with Staggered Wage and Price Dynamics"""
6196,"""Olivier Agard – Kracauer: le chiffonier mélancolique"""
5444,"""At the time of diagnosis, Ph cells from both chronic phase chronic myeloid leukemia and acute lymph…"
6457,"""A strategy to reduce the count of moment conditions in panel data GMM"""
…,…
724333,"""Dal pater familias alla multigenitorialità: cinquanta anni del diritto di famiglia italiano"""
724369,"""Capitolo quarantaduesimo Gli enti pubblici"""
724125,"""The Accelerated Access Initiative to Quality Formal Education for Syrian Refugee Children (AAI). Le…"


In [11]:
from string import Template
from SPARQLWrapper import SPARQLWrapper, CSV, JSON
from SPARQLWrapper.SPARQLExceptions import QueryBadFormed
from urllib.error import HTTPError
from requests import get


sparql = SPARQLWrapper("https://test.opencitations.net/meta/sparql")

findings = []

def get_type(doi):
    HTTP_HEADERS = {"authorization": "8c0f10ec-f033-4e81-a4ec-818a0232c1f8"}
    API_CALL = "https://w3id.org/oc/meta/api/v1/metadata/{}"

    response = get(API_CALL.format('doi:'+doi), headers=HTTP_HEADERS)

    return response.json()[0]['type']

titles_df = pl.DataFrame()


for iris_id, title in tqdm(iris_noid_titles[140:].iter_rows(), total=len(iris_noid_titles[140:])):
    title = title.replace('\r', ' ').replace('\n', '').replace('"', "'")
    if len(title.split()) < 3:
        continue
    try:
        sparql.setQuery(f"""
                        PREFIX datacite: <http://purl.org/spar/datacite/>
                        PREFIX dcterms: <http://purl.org/dc/terms/>
                        PREFIX literal: <http://www.essepuntato.it/2010/06/literalreification/>
                        PREFIX fabio: <http://purl.org/spar/fabio/>
                        SELECT ?entity ?doi ?type
                        WHERE {{
                            ?entity dcterms:title "{title}" ;
                                a ?type.
                            ?entity datacite:hasIdentifier ?identifier.
                            ?identifier datacite:usesIdentifierScheme datacite:doi.
                            ?identifier literal:hasLiteralValue ?doi.
                        FILTER (?type != fabio:Expression)
                        }}""")
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        if results["results"]["bindings"]:
            print(title, results["results"]["bindings"])
            for result in results["results"]["bindings"]:
                entity = result["entity"]["value"]
                doi = result["doi"]["value"]
                type = result["type"]["value"]
                findings.append({'title': title, 'omid': entity.replace("https://w3id.org/oc/meta/", 'omid:'), 'id': "doi:"+doi, 'type': get_type(doi), 'iris_id': iris_id})

    except (QueryBadFormed, HTTPError) as e:
        continue

  0%|          | 0/103341 [00:00<?, ?it/s]

  0%|          | 8/103341 [00:01<4:01:26,  7.13it/s]

Hybrid Modelling Of A Car Driveline For Servo-Actuated Gear Shift [{'entity': {'type': 'uri', 'value': 'https://w3id.org/oc/meta/br/061403332327'}, 'doi': {'type': 'literal', 'value': '10.1109/isie.2005.1528916'}, 'type': {'type': 'uri', 'value': 'http://purl.org/spar/fabio/ProceedingsPaper'}}]


  0%|          | 51/103341 [00:08<4:34:13,  6.28it/s] 


KeyboardInterrupt: 

In [7]:
findings

[{'title': 'Hybrid Modelling Of A Car Driveline For Servo-Actuated Gear Shift',
  'omid': 'omid:br/061403332327',
  'id': 'doi:10.1109/isie.2005.1528916',
  'type': 'proceedings article',
  'iris_id': 55620}]

In [8]:
findings_df = pl.DataFrame(findings)
findings_df

title,omid,id,type,iris_id
str,str,str,str,i64
"""Hybrid Modelling Of A Car Driveline For Servo-Actuated Gear Shift""","""omid:br/061403332327""","""doi:10.1109/isie.2005.1528916""","""proceedings article""",55620


In [10]:
lf_iim = pl.scan_parquet('data/iris_in_meta/*.parquet')
lf_iim.head().collect()

id,title,type,omid,iris_id
str,str,str,str,i32
"""doi:10.1002/widm.1511""","""A White Paper On Good Research Practices In Benchmarking: The Case Of Cluster Analysis""","""journal article""","""omid:br/06510108547""",776472
"""doi:10.1093/annonc/mdx033""","""European Cancer Mortality Predictions For The Year 2017, With Focus On Lung Cancer""","""journal article""","""omid:br/062503328173""",388298
"""doi:10.1016/j.jmatprotec.2017.10.019""","""High Throughput Hybrid Laser Assisted Machining Of Sintered Reaction Bonded Silicon Nitride""","""journal article""","""omid:br/062503328812""",320811
"""doi:10.1093/annonc/mdw306""","""Global Trends And Predictions In Ovarian Cancer Mortality""","""journal article""","""omid:br/062503328364""",632727
"""doi:10.1093/annonc/mdw430""","""Short, Full-Dose Adjuvant Chemotherapy (CT) In High-Risk Adult Soft Tissue Sarcomas (STS): Long-Ter…","""journal article""","""omid:br/062503328314""",301915


#### dask

In [91]:
iris_noid_titles = (
    df[['IDE_DOI', 'IDE_ISBN', 'TITLE']]
    .filter(
        pl.col('IDE_DOI').is_null() & pl.col('IDE_ISBN').is_null()
    )
)['TITLE'].to_list()

In [45]:
len(iris_noid_titles)

105327

In [9]:
zip_file = ZipFile('data/csv_openalex.zip')
csvs = ['zip://'+n for n in zip_file.namelist() if n.endswith('.csv')]

In [13]:
ddf = dd.read_csv(csvs, storage_options={'fo': zip_file.filename}, usecols=['title'])

In [30]:
ddf.partitions[0].head()

[########################################] | 100% Completed | 101.88 ms


,title
0,Expression Of Cyclin-Dependent Kinases Inhibit...
1,Effects Of Treatment Of Rats With Indole-3-Car...
2,Parathyroid Hormone-Related Peptide And Cardio...
3,Oral Immunisation Of Chickens Using Cholera To...
4,"Influence Of Stage Classification, Tumor Diffe..."


In [ ]:
import numpy as np


def compare_title_similarity(title, list_):
    res = process.extractOne(title, list_, scorer=fuzz.token_sort_ratio, score_cutoff=70)
    return title, res


def find_fuzzy(list_from_user, list_from_DB, score_cutoff: int):
    score_matrix = process.cdist(
        list_from_user,
        list_from_DB,
        processor=str.lower,
        scorer=fuzz.ratio,
        dtype=np.uint8,  # Output the score as uint8, which is faster.
        workers=-1,  # Use multithreading. -1 means use all cores.
        score_cutoff=score_cutoff,
    )

    results = []
    user_indices, db_indices = np.nonzero(score_matrix)
    for user_index_of_match, db_index_of_match in zip(user_indices, db_indices):
        results.append(
            {
                "user_index_of_match": user_index_of_match,
                "db_index_of_match": db_index_of_match,
                "user_item_of_match": list_from_user[user_index_of_match],
                "db_item_of_match": list_from_DB[db_index_of_match],
                "score_of_match": score_matrix[user_index_of_match, db_index_of_match],
            }
        )
    return results


In [48]:
#def starts_with(df):


title_ddf = ddf.partitions[0].title.str.startswith('Expression').compute()
title_ddf

AttributeError: 'Series' object has no attribute 'query'

In [158]:
lf_iim_null = pl.scan_parquet('data/iris_in_meta_no_id/*.parquet')

#lf_iim_null.select(pl.len()).collect().head()
#lf_iim_null.collect().head(20)

lf_iim_null = (
    lf_iim_null
    .select('title')
    .filter(
        (pl.col('title') == "").not_()
    )
)

lf_iim_null.select(pl.len()).collect()

len
u32
12352039


In [177]:
116007 + 12352039

12468046

In [159]:
for title in iris_noid_titles:
    matches = process.extract(title, lf_iim_null['title'].to_list(), scorer=fuzz.token_sort_ratio, limit=1)
    if matches:
        print(title, matches[0])

TypeError: 'LazyFrame' object is not subscriptable (aside from slicing)

Use `select()` or `filter()` instead.

In [27]:
def compare_title_similarity(title):
    res = process.extractOne(title, l, scorer=fuzz.token_sort_ratio, score_cutoff=70)
    return title, res

In [5]:


lf_im_null_res = (
    lf_im_null
    .select(['title'])
    .with_columns(
        (pl.col('title').map_elements(compare_title_similarity, return_dtype=pl.Int64)).alias('title_similarity')
    )
    .drop_nulls('title_similarity')
)

NameError: name 'lf_im_null' is not defined

In [11]:
lf_im_null_res = (
    lf_im_null
    .select(['title'])
    .filter(
        pl.col('title') != ""
    )
)

#lf_im_null.select(pl.len()).collect().head()
l = lf_im_null_res.collect()['title']

In [12]:
type(l)

polars.series.series.Series

In [17]:
len(noid_titles_list)

105327

In [22]:
from tqdm.contrib.concurrent import process_map


core_num = multiprocessing.cpu_count()

r = process_map(compare_title_similarity, noid_titles_list[:10], max_workers=core_num, chunksize=1)

print(r)


  0%|          | 0/10 [00:00<?, ?it/s]

[('Applications of CeCl3.7H2O-NaI System Towards the Formation of Heterocyclic Structures', ('[The Interferon System: Structure, Biology, Applications].', 58.33333333333333, 4574416)), ('Radioattività naturale delle materie prime ceramiche e relative condizioni di esposizione occupazionale nell’industria', None), ('Gli istituti generali di semplificazione: la conferenza di servizi e le sue trasformazioni', ('Transformation Consistent Self-ensembling Model For Semi-supervised Medical Image Segmentation', 51.08695652173913, 5867090)), ("Commento all’art.8 d.lgs. 154/2004 - Procedimenti ai sensi dell'articolo 88 del Trattato istitutivo della Comunitaà europea", None), ('2005: agricoltura centro o fine del mondo?', ('Agricultural Policy Monitoring And Evaluation 2020', 54.347826086956516, 5804189)), ('Aspetti emotivi e comportamentali dell’improvvisazione jazzistica: il punto di vista del musicista', None), ('Phaseguide Structures for Pipette Actuated Laminar Flow Based Selective Sample Re

In [23]:
r

[('Applications of CeCl3.7H2O-NaI System Towards the Formation of Heterocyclic Structures',
  ('[The Interferon System: Structure, Biology, Applications].',
   58.33333333333333,
   4574416)),
 ('Radioattività naturale delle materie prime ceramiche e relative condizioni di esposizione occupazionale nell’industria',
  None),
 ('Gli istituti generali di semplificazione: la conferenza di servizi e le sue trasformazioni',
  ('Transformation Consistent Self-ensembling Model For Semi-supervised Medical Image Segmentation',
   51.08695652173913,
   5867090)),
 ("Commento all’art.8 d.lgs. 154/2004 - Procedimenti ai sensi dell'articolo 88 del Trattato istitutivo della Comunitaà europea",
  None),
 ('2005: agricoltura centro o fine del mondo?',
  ('Agricultural Policy Monitoring And Evaluation 2020',
   54.347826086956516,
   5804189)),
 ('Aspetti emotivi e comportamentali dell’improvvisazione jazzistica: il punto di vista del musicista',
  None),
 ('Phaseguide Structures for Pipette Actuated La

In [20]:
noid_titles_list[:10]

['Applications of CeCl3.7H2O-NaI System Towards the Formation of Heterocyclic Structures',
 'Radioattività naturale delle materie prime ceramiche e relative condizioni di esposizione occupazionale nell’industria',
 'Gli istituti generali di semplificazione: la conferenza di servizi e le sue trasformazioni',
 "Commento all’art.8 d.lgs. 154/2004 - Procedimenti ai sensi dell'articolo 88 del Trattato istitutivo della Comunitaà europea",
 '2005: agricoltura centro o fine del mondo?',
 'Aspetti emotivi e comportamentali dell’improvvisazione jazzistica: il punto di vista del musicista',
 'Phaseguide Structures for Pipette Actuated Laminar Flow Based Selective Sample Recovery',
 'Studio del ciclo biologico di Myxobolus lentisuturalis (Myxozoa, Myxobolidae), parassita di Carassius auratus auratus.',
 'The Other Side of the Timing Equation: a Result of Clock Faults',
 'Ritratto di Augusto capite velato']

In [19]:
l[4574416]

'[The Interferon System: Structure, Biology, Applications].'

### RQ1. What is the coverage of the publications available in IRIS (strictly concerning research conducted within the University of Bologna) in OpenCitations Meta?

In [17]:
lf_iim = pl.scan_parquet('data/iris_in_meta/*.parquet')
#lf_iim.head().collect()
print(lf_iim.select(pl.len()).collect())
print(lf_iim.filter(pl.col('id').str.contains('doi')).select(pl.len()).collect().item())
print(lf_iim.filter(pl.col('id').str.contains('pmid')).select(pl.len()).collect().item())
print(lf_iim.filter(pl.col('id').str.contains('isbn')).select(pl.len()).collect().item())

shape: (1, 1)
┌────────┐
│ len    │
│ ---    │
│ u32    │
╞════════╡
│ 117764 │
└────────┘
115776
1078
910


In [54]:
pl.Config.set_fmt_str_lengths(70)
lf_iim.collect().sample(5)

id,title,type,omid,iris_id
str,str,str,str,i32
"""doi:10.1038/s41467-021-26551-x""","""Genome-wide Association Study Identifies Susceptibility Loci For Acut…","""journal article""","""omid:br/061401295311""",788512
"""doi:10.1109/mmm.2016.2525119""","""Smart Solutions In Smart Spaces: Getting The Most From Far-Field Wire…","""journal article""","""omid:br/062602181087""",263669
"""doi:10.1063/1.5007641""","""Results From CUORE And CUORE-0""","""proceedings article""","""omid:br/062502355373""",330526
"""doi:10.1177/1362480619859350""","""Immigration Detention As Social Defence: Policing ‘Dangerous Mobility…","""journal article""","""omid:br/062401318568""",441872
"""doi:10.1016/j.ejso.2022.04.011""","""Trial Sequential Analysis Of Randomized Controlled Trials On Neoadjuv…","""journal article""","""omid:br/061201741385""",684086


In [29]:
from src.read_iris import read_iris

iris = read_iris('./data/iris-data-2024-03-14', not_filtered=True)
iris

ITEM_ID,IDE_DOI,IDE_ISBN,IDE_PMID,YEAR_PUBLISHED,TITLE
i32,str,str,str,str,str
2766,"""10.14411/fp.2008.015""",null,"""18666414""","""2008""","""Bothriocephalidean tapeworms (Cestoda) from the blackfish, Centrolophus niger (Perciformes: Centrol…"
3349,"""10.1016/j.econlet.2011.12.096""",null,null,"""2012""","""Low-Quality leadership in a vertically differentiated duopoly with Cournot competition"""
3867,null,"""9788876655746""",null,"""2009""","""Due autografi inediti di Luigi Dallapiccola"""
4342,null,"""8821724038""",null,"""2006""","""Il sistema informativo contabile. Dalla pianificazione alla rendicontazione dei risultati"""
4585,null,"""9788861328112""",null,"""2012""","""Commento all'art. 1271 cod. civ."""
…,…,…,…,…,…
724623,"""10.3390/vetsci10010045""",null,"""36669046""","""2023""","""Detection of Necrosis in Digitised Whole-Slide Images for Better Grading of Canine Soft-Tissue Sarc…"
724249,null,"""9788898392117""",null,"""2014""","""The Islamic ecumene. Erbil and its region from the conquest of Islam to the present"""
724571,null,null,null,"""2022""","""La Costituzione di Dobbs v. Jackson Women’s Health Organization. Un’analisi critica"""


In [2]:
from src.get_iris_dois_isbns import get_iris_dois_pmids_isbns

dois_isbns_pmids = get_iris_dois_pmids_isbns('data/iris-data-2024-03-14')

In [18]:
rq1 = lf_iim.select(pl.len()).collect()

print(rq1.item(), 'articoli provenienti da IRIS su', len(dois_isbns_pmids), 'sono presenti in OM')
print("{:.1f}%".format(rq1.item() / len(dois_isbns_pmids) * 100))

117764 articoli provenienti da IRIS su 225042 sono presenti in OM
52.3%


In [103]:
ids_list = (
    lf_iim
    .select('id')
)

ids_list.head().collect()

id
str
"""doi:10.1002/widm.1511"""
"""doi:10.1016/j.jmatprotec.2017.10.019"""
"""doi:10.1093/annonc/mdw618"""
"""doi:10.1093/annonc/mdw430"""
"""doi:10.1093/annonc/mdw649"""


### RQ2. Which are the types of publications that are better covered in OpenCitations Meta?

In [150]:
rq2 = lf_iim.group_by('type').len().sort('len', descending=True).with_columns(pl.col('type').str.replace(r"^$", 'no type')).collect()

print(rq2)

shape: (16, 2)
┌─────────────────────┬────────┐
│ type                ┆ len    │
│ ---                 ┆ ---    │
│ str                 ┆ u32    │
╞═════════════════════╪════════╡
│ journal article     ┆ 103900 │
│ proceedings article ┆ 5584   │
│ book chapter        ┆ 4473   │
│ no type             ┆ 1424   │
│ book                ┆ 917    │
│ …                   ┆ …      │
│ dataset             ┆ 6      │
│ dissertation        ┆ 2      │
│ book series         ┆ 1      │
│ series              ┆ 1      │
│ computer program    ┆ 1      │
└─────────────────────┴────────┘


### Index to Parquet

In [15]:
omids_list = (
    lf_iim
    .select('omid')
    .collect()
)['omid'].to_list()

len(omids_list)

116007

In [16]:
index_path = Path('data/oc_index')
file_names = [Path(index_path / archive) for archive in os.listdir(index_path)]

In [27]:
c = 0
for archive in tqdm(file_names):
    print(archive)
    zip_file = ZipFile(archive)

    csvs = ['zip://'+n for n in zip_file.namelist() if n.endswith('.csv')]

    c += len(csvs)
    ddf = dd.read_csv(csvs, storage_options={'fo': zip_file.filename}, usecols=['id', 'citing', 'cited'])
    ddf = ddf[ddf['cited'].isin(omids_list) | ddf['citing'].isin(omids_list)]
    ddf.to_parquet('data/index_in_iris/' + zip_file.filename + '.parquet', write_index=False) # 154 mins

  0%|          | 0/25 [00:00<?, ?it/s]

data/oc_index/2023_08_03-05T_0_1.zip
[########################################] | 100% Completed | 10m 6ss


  4%|▍         | 1/25 [10:06<4:02:40, 606.67s/it]

data/oc_index/2023_11_25T_0_1.zip
[########################################] | 100% Completed | 38.91 s


  8%|▊         | 2/25 [10:46<1:44:40, 273.07s/it]

...

 92%|█████████▏| 23/25 [2:29:30<15:48, 474.25s/it]

data/oc_index/2023_08_24-27T_0_1.zip
[########################################] | 100% Completed | 37.13 s


 96%|█████████▌| 24/25 [2:30:07<05:43, 343.18s/it]

data/oc_index/2023_06_13-14T_0_1-10.zip
[########################################] | 100% Completed | 248.34 s


100%|██████████| 25/25 [2:34:16<00:00, 370.25s/it]


### RQ 3. What is the amount of citations (according to OpenCitations Index) included in the IRIS publications that are involved in OpenCitations Meta (as citing entity and as cited entity)?

In [24]:
iii_path = Path('data/index_in_iris/')

iii_glob = glob.glob(str(iii_path / '*' / '*.parquet'))

In [25]:
lf_iii = pl.scan_parquet(iii_glob)
print(lf_iii.select(pl.len()).collect())
lf_iii.head().collect()

shape: (1, 1)
┌─────────┐
│ len     │
│ ---     │
│ u32     │
╞═════════╡
│ 7859226 │
└─────────┘


id,citing,cited
str,str,str
"""oci:062102010944-06403538172""","""omid:br/062102010944""","""omid:br/06403538172"""
"""oci:062102010936-061901873468""","""omid:br/062102010936""","""omid:br/061901873468"""
"""oci:061302445980-0620582911""","""omid:br/061302445980""","""omid:br/0620582911"""
"""oci:062103078814-061903865447""","""omid:br/062103078814""","""omid:br/061903865447"""
"""oci:06403471424-06703722896""","""omid:br/06403471424""","""omid:br/06703722896"""


In [23]:
oc_omids_list = (
    lf_iim
    .select('omid')
    .collect()
)['omid'].to_list()

len(oc_omids_list)

110931

In [46]:
rq3a = (
    lf_iii
    .select('citing')
    .unique()
    .filter(
        pl.col('citing').is_in(oc_omids_list)
    )
)

citing_list = rq3a.collect()['citing'].to_list()

rq3a.head().collect()
rq3a.select(pl.len()).collect()

len
u32
107704


In [47]:
rq3b = (
    lf_iii
    .select('cited')
    .unique()
    .filter(
        pl.col('cited').is_in(oc_omids_list)
    )
)

cited_list = rq3b.collect()['cited'].to_list()

rq3b.head().collect()
rq3b.select(pl.len()).collect()

len
u32
105135


In [79]:
cit_coming_from_iris = (
    lf_iii
    .filter(
        pl.col('citing').is_in(oc_omids_list)
    )
)

print(cit_coming_from_iris.select(pl.len()).collect())

shape: (1, 1)
┌─────────┐
│ len     │
│ ---     │
│ u32     │
╞═════════╡
│ 4295973 │
└─────────┘


### RQ 4. How many of these citations come from and go to publications that are not included in IRIS?

In [52]:
len(oc_omids_list)

116007

In [26]:
rq4a = (
    lf_iii
    .select('citing')
    #.unique('cited') ???
    .filter(
        ~pl.col('citing').is_in(oc_omids_list)
    )
)

In [27]:
print(rq4a.select(pl.len()).collect())
rq4a.head().collect()

shape: (1, 1)
┌─────────┐
│ len     │
│ ---     │
│ u32     │
╞═════════╡
│ 3830647 │
└─────────┘


citing
str
"""omid:br/062102010944"""
"""omid:br/062102010936"""
"""omid:br/061302445980"""
"""omid:br/062103078814"""
"""omid:br/0660213594"""


In [ ]:
rq4b = (
    lf_iii
    .select('cited')
    .filter(
        ~pl.col('cited').is_in(oc_omids_list)
    )
)

In [116]:
print(rq4b.select(pl.len()).collect())
rq4b.head().collect()

shape: (1, 1)
┌─────────┐
│ len     │
│ ---     │
│ u32     │
╞═════════╡
│ 3939554 │
└─────────┘


cited
str
"""omid:br/06703722896"""
"""omid:br/06703722887"""
"""omid:br/06703722545"""
"""omid:br/06703723039"""
"""omid:br/06803732245"""


In [162]:
rq4 = (
    lf_iii
    .select('cited', 'citing')
    .filter(
        ~pl.col('cited').is_in(oc_omids_list),
        ~pl.col('citing').is_in(oc_omids_list)
    )
)

In [164]:
print(rq4.select(pl.len()).collect())

shape: (1, 1)
┌─────┐
│ len │
│ --- │
│ u32 │
╞═════╡
│ 0   │
└─────┘


### RQ 5. How many of these citations involve publications in IRIS as both citing and cited entities?

In [129]:
rq5a = (
    lf_iii
    .select('citing')
    .filter(
        pl.col('citing').is_in(oc_omids_list)
    )
)

In [130]:
print(rq5a.select(pl.len()).collect())
rq5a.head().collect()

shape: (1, 1)
┌─────────┐
│ len     │
│ ---     │
│ u32     │
╞═════════╡
│ 4295973 │
└─────────┘


citing
str
"""omid:br/06403471424"""
"""omid:br/06403471424"""
"""omid:br/06403471424"""
"""omid:br/06403471424"""
"""omid:br/06403471424"""


In [121]:
rq5b = (
    lf_iii
    .select('cited')
    .filter(
        pl.col('cited').is_in(oc_omids_list)
    )
)

In [122]:
print(rq5b.select(pl.len()).collect())
rq5b.head().collect()

shape: (1, 1)
┌─────────┐
│ len     │
│ ---     │
│ u32     │
╞═════════╡
│ 3919672 │
└─────────┘


cited
str
"""omid:br/06403538172"""
"""omid:br/061901873468"""
"""omid:br/0620582911"""
"""omid:br/061903865447"""
"""omid:br/06502545532"""


In [133]:
rq5 = (
    lf_iii
    .select('citing', 'cited')
    .filter(
        pl.col('citing').is_in(oc_omids_list) & pl.col('cited').is_in(oc_omids_list)
    )
)

In [134]:
print(rq5.select(pl.len()).collect())
rq5.head().collect()

shape: (1, 1)
┌────────┐
│ len    │
│ ---    │
│ u32    │
╞════════╡
│ 356419 │
└────────┘


citing,cited
str,str
"""omid:br/06801833332""","""omid:br/06301833698"""
"""omid:br/06801833332""","""omid:br/06201835146"""
"""omid:br/06801833332""","""omid:br/06101850825"""
"""omid:br/06140651081""","""omid:br/061301899117"""
"""omid:br/061902028230""","""omid:br/06230928593"""


---

### Further usage

In [30]:
from src.read_iris_in_meta import read_iris_in_meta

iim_df = read_iris_in_meta()
iim_df.collect()

id,title,type,omid,iris_id
str,str,str,str,i32
"""doi:10.1002/widm.1511""","""A White Paper On Good Research Practices In Benchmarking: The Case Of Cluster Analysis""","""journal article""","""omid:br/06510108547""",776472
"""doi:10.1093/annonc/mdx033""","""European Cancer Mortality Predictions For The Year 2017, With Focus On Lung Cancer""","""journal article""","""omid:br/062503328173""",388298
"""doi:10.1016/j.jmatprotec.2017.10.019""","""High Throughput Hybrid Laser Assisted Machining Of Sintered Reaction Bonded Silicon Nitride""","""journal article""","""omid:br/062503328812""",320811
"""doi:10.1093/annonc/mdw306""","""Global Trends And Predictions In Ovarian Cancer Mortality""","""journal article""","""omid:br/062503328364""",632727
"""doi:10.1093/annonc/mdw430""","""Short, Full-Dose Adjuvant Chemotherapy (CT) In High-Risk Adult Soft Tissue Sarcomas (STS): Long-Ter…","""journal article""","""omid:br/062503328314""",301915
…,…,…,…,…
"""doi:10.1002/sim.7247""","""Relational Event Models For Longitudinal Network Data With An Application To Interhospital Patient …","""journal article""","""omid:br/0690743876""",311099
"""doi:10.1016/j.bmc.2004.08.012""","""Effects Of New Ubiquinone-Imidazo[2,1-B]Thiazoles On Mitochondrial Complex I (NADH-ubiquinone Reduc…","""journal article""","""omid:br/0690748197""",54217
"""doi:10.1016/j.nonrwa.2010.10.003""","""Nonhypoellipticity And Comparison Principle For Partial Differential Equations Of Black–Scholes Typ…","""journal article""","""omid:br/0690748601""",130283


In [8]:
iris_df

ITEM_ID,IDE_DOI,IDE_ISBN,IDE_PMID
i32,str,str,str
2766,"""10.14411/fp.20…",null,"""18666414"""
3349,"""10.1016/j.econ…",null,null
3867,null,"""9788876655746""",null
4342,null,"""8821724038""",null
4585,null,"""9788861328112""",null
…,…,…,…
724609,"""10.1007/s11244…",null,"""36405974"""
724546,null,"""9788859622994""",null
724623,"""10.3390/vetsci…",null,"""36669046"""
